# Check reliability problems reference probabilities

In this example, we check that the reference probabilities in the reliability problems are consistent with confidence bounds from Monte-Carlo simulations. These 95% confidence bounds are stored in 'reliability_compute_reference_proba.csv' and required approximately than $10^9$ function evaluations for each problem. 

We consider two different metrics:
* we check if the reference probability is within the 95% confidence bounds,
* we compute the number of significant digits by comparing the Monte-Carlo estimator and the reference value.

The number of significant digits may be as high as 17 when all decimal digits are correct. However, the reference probabilities are only known up to 3 digits for most problems. In order to keep some safeguard, we will be happy with 2 correct digits.

These metrics may fail.
* On average, a fraction equal to 5% of the estimates are outside the confidence bounds. 
* The Monte-Carlo estimator may not be accurate enough, e.g. if the probability is very close to zero.

In [1]:
import openturns as ot
import otbenchmark as otb
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("reliability_compute_reference_proba.csv")
df

,Unnamed: 0,PF,N. function calls,PMin,PMax,C.O.V.,Digits,Time (s)
0,RP8,7.908179e-04,2.413400e+08,7.872713e-04,7.943646e-04,0.002288,1.640503,300.011341
1,RP14,7.708905e-04,7.441900e+08,7.688964e-04,7.728846e-04,0.001320,1.879484,300.004585
2,RP22,4.207357e-03,1.495610e+09,4.204076e-03,4.210637e-03,0.000398,2.400308,300.000947
3,RP24,2.860848e-03,1.644700e+09,2.858266e-03,2.863429e-03,0.000460,2.336891,300.001723
4,RP25,4.175883e-05,1.567860e+09,4.143895e-05,4.207871e-05,0.003908,1.408015,300.015279
5,RP28,1.315725e-07,1.839290e+09,1.149947e-07,1.481503e-07,0.064286,0.191886,300.000353
6,RP31,3.227556e-03,1.787260e+09,3.224926e-03,3.230186e-03,0.000416,2.381211,300.000409
7,RP33,2.574817e-03,1.430700e+09,2.572190e-03,2.577443e-03,0.000520,2.283686,300.000557
8,RP35,3.478964e-03,1.415140e+09,3.475896e-03,3.482032e-03,0.000450,2.346860,300.002611
9,RP38,8.059349e-03,7.765700e+08,8.053061e-03,8.065638e-03,0.000398,2.399976,300.003572


In [3]:
data = df.values

In [4]:
pf_reference = data[:, 1]
pmin = data[:, 3]
pmax = data[:, 4]

In [5]:
benchmarkProblemList = otb.ReliabilityBenchmarkProblemList()
numberOfProblems = len(benchmarkProblemList)
numberOfProblems

26

In [6]:
digitsMinimum = 2

In [7]:
categoryA = []
categoryB = []
categoryC = []
categoryD = []

In [8]:
for i in range(numberOfProblems):
    problem = benchmarkProblemList[i]
    name = problem.getName()
    pf = problem.getProbability()
    event = problem.getEvent()
    antecedent = event.getAntecedent()
    distribution = antecedent.getDistribution()
    dimension = distribution.getDimension()
    if pf > pmin[i] and pf < pmax[i]:
        tagBounds = "In"
    else:
        tagBounds = "Out"
    digits = otb.ComputeLogRelativeError(pf_reference[i], pf)
    if tagBounds == "In" and digits >= digitsMinimum:
        categoryA.append(name)
    elif tagBounds == "Out" and digits >= digitsMinimum:
        categoryB.append(name)
    elif tagBounds == "In" and digits < digitsMinimum:
        categoryC.append(name)
    else:
        categoryD.append(name)
    print(
        "#%d, %-10s, pf=%.2e, ref=%.2e, C.I.=[%.2e,%.2e], digits=%d : %s"
        % (i, name[0:10], pf, pf_reference[i], pmin[i], pmax[i], digits, tagBounds)
    )

#0, RP8       , pf=7.84e-04, ref=7.91e-04, C.I.=[7.87e-04,7.94e-04], digits=2 : Out
#1, RP14      , pf=7.52e-03, ref=7.71e-04, C.I.=[7.69e-04,7.73e-04], digits=0 : Out
#2, RP22      , pf=4.16e-03, ref=4.21e-03, C.I.=[4.20e-03,4.21e-03], digits=1 : Out
#3, RP24      , pf=2.86e-03, ref=2.86e-03, C.I.=[2.86e-03,2.86e-03], digits=3 : In
#4, RP25      , pf=6.14e-06, ref=4.18e-05, C.I.=[4.14e-05,4.21e-05], digits=0 : Out
#5, RP28      , pf=1.46e-07, ref=1.32e-07, C.I.=[1.15e-07,1.48e-07], digits=0 : In
#6, RP31      , pf=1.80e-04, ref=3.23e-03, C.I.=[3.22e-03,3.23e-03], digits=0 : Out
#7, RP33      , pf=2.57e-03, ref=2.57e-03, C.I.=[2.57e-03,2.58e-03], digits=2 : Out
#8, RP35      , pf=3.54e-03, ref=3.48e-03, C.I.=[3.48e-03,3.48e-03], digits=1 : Out
#9, RP38      , pf=8.10e-03, ref=8.06e-03, C.I.=[8.05e-03,8.07e-03], digits=2 : Out
#10, RP53      , pf=3.13e-02, ref=3.13e-02, C.I.=[3.13e-02,3.13e-02], digits=3 : Out
#11, RP55      , pf=5.60e-01, ref=5.60e-01, C.I.=[5.60e-01,5.60e-01], digits=

There are four different cases.
* Category A: all good. For some problems, both metrics are correct in the sense that the reference probability is within the bounds and the number of significant digits is larger than 2. The RP24, RP55, RP110, RP63, R-S, Axial stressed beam problems fall in that category.
* Category B: correct digits, not in bounds. We see that the RP8 problem has a reference probability outside of the 95% confidence bounds, but has 2 significant digits.
* Category C: insufficient digits, in bounds. The difficult RP28 problem fall in that category.
* Category D: insufficient digits, not in bounds. These are suspicious problems.

In [9]:
print(categoryA)

['RP24', 'RP55', 'RP110', 'RP63', 'R-S', 'Axial stressed beam']


In [10]:
print(categoryB)

['RP8', 'RP33', 'RP38', 'RP53', 'RP54', 'RP57', 'RP89', 'RP91']


In [11]:
print(categoryC)

['RP28', 'RP107', 'RP111', 'RP77']


In [12]:
print(categoryD)

['RP14', 'RP22', 'RP25', 'RP31', 'RP35', 'RP75', 'RP60', 'Four-branch serial system']


The number of suspicious problems seems very large. However, we notice that all these cases are so that the reference probability is close, in absolute value, to the Monte-Carlo estimator.